In [ ]:
# Importaciones necesarias
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import os

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")

# Configuración de GPU (si está disponible)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"GPU detectadas: {len(gpus)}")
    except RuntimeError as e:
        print(e)
else:
    print("Usando CPU para el entrenamiento")

# Configurar para reproducibilidad
tf.random.set_seed(42)
np.random.seed(42)


In [ ]:
# Parámetros del modelo
IMG_SIZE = (64, 64)  # Tamaño más pequeño para CNN desde cero
CHANNELS = 3         # RGB
INPUT_SHAPE = IMG_SIZE + (CHANNELS,)  # (64, 64, 3)

BATCH_SIZE = 32
EPOCHS = 100         # Más épocas ya que entrenamos desde cero
LEARNING_RATE = 0.001

# Parámetros de regularización
DROPOUT_RATE = 0.5
L2_REG = 1e-4

# Estructura de datos esperada (misma que antes)
GENDER_TRAIN_DIR = "dataset/gender/train"
GENDER_VAL_DIR = "dataset/gender/validation"
AGE_TRAIN_DIR = "dataset/age/train"
AGE_VAL_DIR = "dataset/age/validation"

# Clases
GENDER_CLASSES = ['female', 'male']
AGE_CLASSES = ['0-2', '4-6', '8-12', '15-20', '25-32', '38-43', '48-53', '60-100']

NUM_GENDER_CLASSES = len(GENDER_CLASSES)
NUM_AGE_CLASSES = len(AGE_CLASSES)

print(f"Tamaño de entrada: {INPUT_SHAPE}")
print(f"Clases de género: {GENDER_CLASSES} ({NUM_GENDER_CLASSES})")
print(f"Clases de edad: {AGE_CLASSES} ({NUM_AGE_CLASSES})")
print(f"Épocas de entrenamiento: {EPOCHS}")
print(f"Tasa de aprendizaje: {LEARNING_RATE}")


In [ ]:
def conv_block(x, filters, kernel_size=(3, 3), strides=(1, 1), padding='same', 
               use_batch_norm=True, dropout_rate=0.0, pool_size=(2, 2), 
               activation='relu', name_prefix='conv_block'):
    """
    Crea un bloque convolucional completo con las mejores prácticas
    
    Args:
        x: Tensor de entrada
        filters: Número de filtros de la convolución
        kernel_size: Tamaño del kernel de convolución
        strides: Paso de la convolución
        padding: Tipo de padding
        use_batch_norm: Si usar Batch Normalization
        dropout_rate: Tasa de dropout (0 = sin dropout)
        pool_size: Tamaño del MaxPooling
        activation: Función de activación
        name_prefix: Prefijo para nombres de capas
    
    Returns:
        Tensor procesado
    """
    
    # Capa convolucional con regularización L2
    x = layers.Conv2D(
        filters=filters,
        kernel_size=kernel_size,
        strides=strides,
        padding=padding,
        kernel_regularizer=l2(L2_REG),
        name=f'{name_prefix}_conv'
    )(x)
    
    # Batch Normalization (opcional)
    if use_batch_norm:
        x = layers.BatchNormalization(name=f'{name_prefix}_bn')(x)
    
    # Función de activación
    x = layers.Activation(activation, name=f'{name_prefix}_activation')(x)
    
    # MaxPooling para reducir dimensiones
    if pool_size:
        x = layers.MaxPooling2D(
            pool_size=pool_size, 
            name=f'{name_prefix}_maxpool'
        )(x)
    
    # Dropout para regularización (opcional)
    if dropout_rate > 0:
        x = layers.Dropout(
            dropout_rate, 
            name=f'{name_prefix}_dropout'
        )(x)
    
    return x

print("Función de bloque convolucional definida")


In [ ]:
def create_custom_cnn(input_shape, num_classes, model_name="custom_cnn"):
    """
    Crea una CNN completamente personalizada desde cero
    
    Arquitectura:
    - Entrada: 64x64x3
    - Conv Block 1: 32 filtros → 32x32x32
    - Conv Block 2: 64 filtros → 16x16x64  
    - Conv Block 3: 128 filtros → 8x8x128
    - Conv Block 4: 256 filtros → 4x4x256
    - Clasificador: Flatten → Dense(512) → Dense(num_classes)
    
    Args:
        input_shape: Forma de entrada (altura, ancho, canales)
        num_classes: Número de clases de salida
        model_name: Nombre del modelo
    
    Returns:
        Modelo de Keras compilado
    """
    
    # Entrada
    inputs = layers.Input(shape=input_shape, name='input_image')
    
    print(f"📥 Entrada: {input_shape}")
    
    # =============== BLOQUE CONVOLUCIONAL 1 ===============
    # Detecta características básicas (bordes, texturas simples)
    x = conv_block(
        x=inputs,
        filters=32,              # 32 filtros
        kernel_size=(3, 3),
        dropout_rate=0.1,        # Dropout ligero al principio
        name_prefix='block1'
    )
    print(f"🔲 Block 1: {x.shape[1:]} - Detecta bordes y texturas básicas")
    
    # =============== BLOQUE CONVOLUCIONAL 2 =============== 
    # Detecta formas más complejas (ojos, nariz, boca)
    x = conv_block(
        x=x,
        filters=64,              # Más filtros para más características
        kernel_size=(3, 3),
        dropout_rate=0.2,
        name_prefix='block2'
    )
    print(f"🔲 Block 2: {x.shape[1:]} - Detecta formas faciales básicas")
    
    # =============== BLOQUE CONVOLUCIONAL 3 ===============
    # Detecta patrones faciales más específicos
    x = conv_block(
        x=x,
        filters=128,             # Aún más filtros
        kernel_size=(3, 3),
        dropout_rate=0.3,
        name_prefix='block3'
    )
    print(f"🔲 Block 3: {x.shape[1:]} - Detecta patrones faciales complejos")
    
    # =============== BLOQUE CONVOLUCIONAL 4 ===============
    # Detecta características de alto nivel (expresiones, rasgos distintivos)
    x = conv_block(
        x=x,
        filters=256,             # Máximo número de filtros
        kernel_size=(3, 3),
        dropout_rate=0.4,
        name_prefix='block4'
    )
    print(f"🔲 Block 4: {x.shape[1:]} - Detecta características de alto nivel")
    
    # =============== CLASIFICADOR ===============
    # Convertir feature maps a vector 1D
    x = layers.Flatten(name='flatten')(x)
    print(f"📏 Flatten: {x.shape[1:]} - Convierte a vector 1D")
    
    # Capa densa intermedia con regularización
    x = layers.Dense(
        512, 
        activation='relu',
        kernel_regularizer=l2(L2_REG),
        name='dense_features'
    )(x)
    x = layers.BatchNormalization(name='dense_bn')(x)
    x = layers.Dropout(DROPOUT_RATE, name='dense_dropout')(x)
    print(f"🧠 Dense: 512 neuronas - Combina características")
    
    # Capa de salida
    activation = 'softmax' if num_classes > 2 else 'sigmoid'
    outputs = layers.Dense(
        num_classes,
        activation=activation,
        name='predictions'
    )(x)
    print(f"🎯 Salida: {num_classes} clases con activación {activation}")
    
    # Crear modelo
    model = models.Model(inputs=inputs, outputs=outputs, name=model_name)
    
    return model

print("Función de arquitectura CNN personalizada definida")


In [ ]:
# Importaciones necesarias
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import numpy as np
import matplotlib.pyplot as plt
import os

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")

# Configuración de GPU (si está disponible)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"GPU detectadas: {len(gpus)}")
    except RuntimeError as e:
        print(e)
else:
    print("Usando CPU para el entrenamiento")


In [ ]:
# Parámetros del modelo
IMG_SIZE = (224, 224)  # Tamaño de entrada para MobileNetV2
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 0.001

# Estructura de datos esperada para el entrenamiento
# Los datos deberían estar organizados así:
# 
# dataset/
# ├── gender/
# │   ├── train/
# │   │   ├── male/
# │   │   └── female/
# │   └── validation/
# │       ├── male/
# │       └── female/
# └── age/
#     ├── train/
#     │   ├── 0-2/
#     │   ├── 4-6/
#     │   ├── 8-12/
#     │   ├── 15-20/
#     │   ├── 25-32/
#     │   ├── 38-43/
#     │   ├── 48-53/
#     │   └── 60-100/
#     └── validation/
#         ├── 0-2/
#         ├── 4-6/
#         ├── 8-12/
#         ├── 15-20/
#         ├── 25-32/
#         ├── 38-43/
#         ├── 48-53/
#         └── 60-100/

# Rutas de los datos
GENDER_TRAIN_DIR = "dataset/gender/train"
GENDER_VAL_DIR = "dataset/gender/validation"
AGE_TRAIN_DIR = "dataset/age/train"
AGE_VAL_DIR = "dataset/age/validation"

# Clases
GENDER_CLASSES = ['female', 'male']  # 0: female, 1: male
AGE_CLASSES = ['0-2', '4-6', '8-12', '15-20', '25-32', '38-43', '48-53', '60-100']

NUM_GENDER_CLASSES = len(GENDER_CLASSES)
NUM_AGE_CLASSES = len(AGE_CLASSES)

print(f"Clases de género: {GENDER_CLASSES}")
print(f"Clases de edad: {AGE_CLASSES}")
print(f"Número de clases de género: {NUM_GENDER_CLASSES}")
print(f"Número de clases de edad: {NUM_AGE_CLASSES}")


In [ ]:
# Generadores de datos con aumento de datos para entrenamiento
train_datagen = ImageDataGenerator(
    rescale=1./255,              # Normalización de píxeles a [0,1]
    rotation_range=20,           # Rotación aleatoria hasta 20 grados
    width_shift_range=0.2,       # Desplazamiento horizontal
    height_shift_range=0.2,      # Desplazamiento vertical
    shear_range=0.2,             # Transformación de corte
    zoom_range=0.2,              # Zoom aleatorio
    horizontal_flip=True,        # Volteo horizontal
    fill_mode='nearest'          # Relleno de píxeles
)

# Generador de datos para validación (solo normalización)
val_datagen = ImageDataGenerator(rescale=1./255)

def create_data_generators(train_dir, val_dir, target_size, batch_size, class_mode='categorical'):
    """
    Crea generadores de datos para entrenamiento y validación
    
    Args:
        train_dir: Directorio de datos de entrenamiento
        val_dir: Directorio de datos de validación
        target_size: Tamaño objetivo de las imágenes
        batch_size: Tamaño del lote
        class_mode: Tipo de clasificación ('categorical' o 'binary')
    
    Returns:
        train_generator, validation_generator
    """
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=class_mode,
        shuffle=True
    )
    
    validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=class_mode,
        shuffle=False
    )
    
    return train_generator, validation_generator

print("Generadores de datos configurados correctamente")


In [ ]:
def create_base_model(input_shape, num_classes, model_name="model"):
    """
    Crea un modelo usando Transfer Learning con MobileNetV2
    
    Args:
        input_shape: Forma de entrada (altura, ancho, canales)
        num_classes: Número de clases para la clasificación
        model_name: Nombre del modelo
    
    Returns:
        modelo compilado
    """
    
    # Cargar MobileNetV2 pre-entrenado sin la capa de clasificación final
    base_model = MobileNetV2(
        weights='imagenet',           # Pesos pre-entrenados en ImageNet
        include_top=False,            # No incluir la capa de clasificación final
        input_shape=input_shape       # Forma de entrada
    )
    
    # Congelar las capas del modelo base para no entrenarlas
    base_model.trainable = False
    
    # Crear el modelo secuencial
    model = keras.Sequential([
        # Modelo base pre-entrenado
        base_model,
        
        # Capa de pooling global para reducir dimensiones
        layers.GlobalAveragePooling2D(),
        
        # Capa de Dropout para regularización
        layers.Dropout(0.2),
        
        # Capa densa intermedia con activación ReLU
        layers.Dense(128, activation='relu'),
        
        # Otra capa de Dropout
        layers.Dropout(0.5),
        
        # Capa de salida
        layers.Dense(num_classes, 
                    activation='softmax' if num_classes > 2 else 'sigmoid',
                    name='predictions')
    ], name=model_name)
    
    return model

print("Función de creación de modelo definida")


In [ ]:
# Crear modelo para clasificación de género (binaria)
input_shape = IMG_SIZE + (3,)  # (224, 224, 3)

print("=== MODELO DE GÉNERO ===")
gender_model = create_base_model(
    input_shape=input_shape,
    num_classes=NUM_GENDER_CLASSES,
    model_name="gender_model"
)

# Compilar modelo de género
gender_model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss='categorical_crossentropy',  # Para clasificación multiclase
    metrics=['accuracy', 'precision', 'recall']
)

print("\\nResumen del modelo de género:")
gender_model.summary()

print("\\n" + "="*50)
print("=== MODELO DE EDAD ===")

# Crear modelo para clasificación de edad (multiclase)
age_model = create_base_model(
    input_shape=input_shape,
    num_classes=NUM_AGE_CLASSES,
    model_name="age_model"
)

# Compilar modelo de edad
age_model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss='categorical_crossentropy',  # Para clasificación multiclase
    metrics=['accuracy', 'precision', 'recall']
)

print("\\nResumen del modelo de edad:")
age_model.summary()


In [ ]:
def get_callbacks(model_name):
    """
    Crea una lista de callbacks para el entrenamiento
    
    Args:
        model_name: Nombre del modelo para guardar los checkpoints
    
    Returns:
        Lista de callbacks
    """
    
    callbacks = [
        # Guardar el mejor modelo basado en la precisión de validación
        ModelCheckpoint(
            filepath=f'best_{model_name}.h5',
            monitor='val_accuracy',
            save_best_only=True,
            save_weights_only=False,
            mode='max',
            verbose=1
        ),
        
        # Parar el entrenamiento si no mejora en 10 épocas
        EarlyStopping(
            monitor='val_accuracy',
            patience=10,
            restore_best_weights=True,
            verbose=1
        ),
        
        # Reducir la tasa de aprendizaje si no mejora en 5 épocas
        ReduceLROnPlateau(
            monitor='val_accuracy',
            factor=0.2,
            patience=5,
            min_lr=1e-7,
            verbose=1
        )
    ]
    
    return callbacks

# Crear callbacks para ambos modelos
gender_callbacks = get_callbacks('model_gender')
age_callbacks = get_callbacks('model_age')

print("Callbacks configurados correctamente")


In [ ]:
# Esta es una demostración del código de entrenamiento
# Para ejecutarlo realmente, necesitas tener los datos organizados

"""
# Crear generadores de datos para género
gender_train_gen, gender_val_gen = create_data_generators(
    GENDER_TRAIN_DIR, 
    GENDER_VAL_DIR, 
    IMG_SIZE, 
    BATCH_SIZE, 
    'categorical'
)

print(f"Clases encontradas para género: {gender_train_gen.class_indices}")

# Entrenar el modelo de género
history_gender = gender_model.fit(
    gender_train_gen,
    steps_per_epoch=gender_train_gen.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=gender_val_gen,
    validation_steps=gender_val_gen.samples // BATCH_SIZE,
    callbacks=gender_callbacks,
    verbose=1
)

# Guardar el modelo final
gender_model.save('model_gender.h5')
print("Modelo de género guardado como 'model_gender.h5'")
"""

print("Código de entrenamiento de género preparado (comentado para demostración)")


In [ ]:
"""
# Crear generadores de datos para edad
age_train_gen, age_val_gen = create_data_generators(
    AGE_TRAIN_DIR, 
    AGE_VAL_DIR, 
    IMG_SIZE, 
    BATCH_SIZE, 
    'categorical'
)

print(f"Clases encontradas para edad: {age_train_gen.class_indices}")

# Entrenar el modelo de edad
history_age = age_model.fit(
    age_train_gen,
    steps_per_epoch=age_train_gen.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=age_val_gen,
    validation_steps=age_val_gen.samples // BATCH_SIZE,
    callbacks=age_callbacks,
    verbose=1
)

# Guardar el modelo final
age_model.save('model_age.h5')
print("Modelo de edad guardado como 'model_age.h5'")
"""

print("Código de entrenamiento de edad preparado (comentado para demostración)")


In [ ]:
def plot_training_history(history, model_name):
    """
    Visualiza el historial de entrenamiento
    
    Args:
        history: Historial de entrenamiento de Keras
        model_name: Nombre del modelo para el título
    """
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle(f'Métricas de Entrenamiento - {model_name}', fontsize=16)
    
    # Precisión
    axes[0, 0].plot(history.history['accuracy'], label='Entrenamiento')
    axes[0, 0].plot(history.history['val_accuracy'], label='Validación')
    axes[0, 0].set_title('Precisión del Modelo')
    axes[0, 0].set_xlabel('Época')
    axes[0, 0].set_ylabel('Precisión')
    axes[0, 0].legend()
    axes[0, 0].grid(True)
    
    # Pérdida
    axes[0, 1].plot(history.history['loss'], label='Entrenamiento')
    axes[0, 1].plot(history.history['val_loss'], label='Validación')
    axes[0, 1].set_title('Pérdida del Modelo')
    axes[0, 1].set_xlabel('Época')
    axes[0, 1].set_ylabel('Pérdida')
    axes[0, 1].legend()
    axes[0, 1].grid(True)
    
    # Precisión (Precision)
    axes[1, 0].plot(history.history['precision'], label='Entrenamiento')
    axes[1, 0].plot(history.history['val_precision'], label='Validación')
    axes[1, 0].set_title('Precisión (Precision)')
    axes[1, 0].set_xlabel('Época')
    axes[1, 0].set_ylabel('Precisión')
    axes[1, 0].legend()
    axes[1, 0].grid(True)
    
    # Recall
    axes[1, 1].plot(history.history['recall'], label='Entrenamiento')
    axes[1, 1].plot(history.history['val_recall'], label='Validación')
    axes[1, 1].set_title('Recall')
    axes[1, 1].set_xlabel('Época')
    axes[1, 1].set_ylabel('Recall')
    axes[1, 1].legend()
    axes[1, 1].grid(True)
    
    plt.tight_layout()
    plt.show()

print("Función de visualización definida")


In [ ]:
"""
# Después del entrenamiento, visualizar los resultados:

# Para el modelo de género:
plot_training_history(history_gender, "Clasificador de Género")

# Para el modelo de edad:
plot_training_history(history_age, "Clasificador de Edad")

# También puedes obtener métricas finales:
print("=== MÉTRICAS FINALES ===")
print(f"Precisión final del modelo de género: {max(history_gender.history['val_accuracy']):.4f}")
print(f"Precisión final del modelo de edad: {max(history_age.history['val_accuracy']):.4f}")
"""

print("Código de visualización preparado")


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

def evaluate_model(model, test_generator, class_names, model_name):
    """
    Evalúa el modelo en datos de prueba y muestra métricas detalladas
    
    Args:
        model: Modelo entrenado
        test_generator: Generador de datos de prueba
        class_names: Nombres de las clases
        model_name: Nombre del modelo
    """
    
    # Predicciones
    test_generator.reset()
    predictions = model.predict(test_generator)
    predicted_classes = np.argmax(predictions, axis=1)
    
    # Etiquetas verdaderas
    true_classes = test_generator.classes
    
    # Reporte de clasificación
    print(f"\\n=== REPORTE DE CLASIFICACIÓN - {model_name} ===")
    print(classification_report(true_classes, predicted_classes, target_names=class_names))
    
    # Matriz de confusión
    cm = confusion_matrix(true_classes, predicted_classes)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names)
    plt.title(f'Matriz de Confusión - {model_name}')
    plt.xlabel('Predicción')
    plt.ylabel('Etiqueta Real')
    plt.show()
    
    # Métricas adicionales
    accuracy = np.sum(predicted_classes == true_classes) / len(true_classes)
    print(f"\\nPrecisión en datos de prueba: {accuracy:.4f}")
    
    return accuracy

print("Función de evaluación definida")
